## Starting Assumptions

We're going to model the cost of one specific home, and compare it to an equivalent rent. [The house is here](http://www.zillow.com/homes/for_sale/fsba_lt/24704738_zpid/12m_days/42.274213,-83.749015,42.251822,-83.794935_rect/14_zm/0_mmm/). I live in this area, and represent that the asking price, Zillow rent estimate, and financing terms are all reasonable.

So here's some rent estimates:

In [1]:
rent_per_month = 1650
rent_increase_per_month = 50.0 / 12

Zillow has also helpfully calculated the 30-year fixed mortgage terms:

In [2]:
price = 232000
down = 46400
apr = 0.03286
term = 12 * 30
yearly_taxes = 3317

## MATH TIME

First, the fundamental law of finance. Net present value of any sum of money:

In [3]:
def npv(future_value, discount_rate, t):
    return future_value / (1 + discount_rate) ** t

Calculating the net present value of all rental costs is fairly simple. Assume we're paying one month's rent as a deposit, and discount each month's rent:

In [4]:
def rent(months, r):
    def cost_of_month(t):
        return rent_per_month + (rent_increase_per_month * t)
    deposit = rent_per_month
    deposit_back = npv(deposit, r, months)
    return -deposit + sum(-npv(cost_of_month(t), r, t) for t in xrange(0, months)) + deposit_back

So, after 10 years, we will pay:

In [5]:
rent(12 * 10, 0)

-227750.0

228k, undiscounted. Sounds about right.

Calculating the cost of buying is obviously more involved. Let's start with some basic mortgage definitions:

In [6]:
class Mortgage:
    def __init__(self, down, total, payments, rate):
        self.down, self.total, self.payments, self.rate = down, total, payments, rate
        
    def monthly_payment(self):
        r = self.rate
        P = self.total - self.down
        N = self.payments
        # From https://en.wikipedia.org/wiki/Mortgage_calculator#Monthly_payment_formula
        return (r * P)  / (1 - (1 + r) ** -N)
        
    def remaining(self, t):
        balance = self.total - self.down
        for _ in xrange(0, t):
            interest = balance * self.rate
            balance -= (self.monthly_payment() - interest)
        return max(0, balance)

And plug in the terms Zillow gave us:

In [7]:
mortgage = Mortgage(down, price, term, apr / 12)

Now let's do some basic sanity checking:

In [8]:
mortgage.monthly_payment()

811.4145714967989

In [9]:
mortgage.remaining(12 * 30)

0

In [10]:
mortgage.remaining(0)

185600

All looking good.

Now we can model the cost of buying. We need to pay:

* money down for the mortgage
* closing costs on purchase, including agent fees, inspection, etc
* closing costs on sale
* monthly maintenance, monthly mortgage
* taxes (there are actually some important tax deductions you can claim against mortgage interest and PMI, but we're going to ignore those for simplicity)

In [11]:
def buy_in_general(mortgage, buy_closing_costs, sale_closing_costs, maintenance, taxes, months, appreciation, r):
    cost_to_close = mortgage.down + (mortgage.total * buy_closing_costs)
    
    monthly_maintenance = maintenance * mortgage.total / 12
    monthly = sum(npv(mortgage.monthly_payment() + monthly_maintenance, r, t) for t in xrange(0, months))
    
    taxes = sum(npv(taxes, r, year * 12) for year in xrange(months / 12))
    
    owed_bank = mortgage.remaining(months)
    from_buyer = mortgage.total * appreciation
    cash_at_sale = from_buyer - owed_bank - (from_buyer * sale_closing_costs)
    
    return -cost_to_close - monthly - taxes + npv(cash_at_sale, r, months)

Let's assume:

* Closing costs are 2% on buying, and 6.5% on sale. Costs are dramatically higher on sale because agent commissions are "paid" by the seller. 
* Maintenance is 1% per year.
* The home does not appreciate in value whatsoever during the term of the investment (which is arguably pessimistic).

This last assumption is by far the most important. It will drastically affect our IRR. We'll get back to that later.

In [12]:
def buy(months, r): 
    return buy_in_general(mortgage, 0.02, 0.065, 0.01, yearly_taxes, months, 1.0, r)

Let's see how much this house will cost us, undiscounted, over several years:

In [13]:
[buy(y * 12, 0) for y in xrange(1, 11)]

[-31400.519055983226,
 -42957.826478991454,
 -54387.811997421464,
 -65686.22822329373,
 -76848.68607812605,
 -87870.65006621715,
 -98747.43339024982,
 -109474.19290395285,
 -120045.92389638702,
 -130457.45470223745]

Let's compare that to renting:

In [14]:
[rent(y * 12, 0) for y in xrange(1, 11)]

[-20075.0,
 -40750.0,
 -62025.0,
 -83900.0,
 -106375.0,
 -129450.0,
 -153125.0,
 -177400.0,
 -202275.0,
 -227750.0]

So, renting becomes more expensive quickly, if we assume our money has no time value. That's not a good assumption, so let's delve a little deeper.

## IRR

IRR is a common concept when evaluating investments. The IRR essentially states the smallest time value your money needs to have for you to lose money on your investment. In other words, you want to know the rate that causes your discounted cash flow to become negative.

In [15]:
def irr(dcf):
    if dcf(0) < 0: return
    for ix in xrange(1000):
        r_t = float(ix) / 1000
        if dcf(r_t) < 0:
            return r_t

Note that there are cases where there is **no** IRR. Your investment loses money, even assuming your upfront dollars are valued equally to later hypothetical dollars. It cannot grow your wealth.

There will be cases where this happens with our house investment.

We can model this "investment" by subtracting the rent we're not paying from the other costs that we now must pay. Now, (drumroll please), let's look at the IRR for our hypothetical house, depending on how long we stay in it:

In [16]:
for year in xrange(1, 16):
    def dcf(r_i): return buy(12 * year, r_i / 12) - rent(12 * year, r_i / 12)
    print year, irr(dcf)

1 None
2 None
3 0.055
4 0.093
5 0.115
6 0.129
7 0.138
8 0.145
9 0.149
10 0.153
11 0.155
12 0.157
13 0.159
14 0.16
15 0.161


Not bad. We're well above the risk free rate in year three, and above expected real returns in most other volatile assets soon after that.

There's obviously a couple caveats. First, all of this analysis is predicated on the house being a principle residence. There are many benefits, especially regarding capital gains, that flow from the government in this case. Investment properties are a whole 'nother ball of wax.

And finally, let's return to the most important factor. Houses can drastically change in value.

## RUH ROH

Everyone who owned a house before the 2008 crash can testify that house values are not stable. So let's look at the worst case, and assume that our house halves in value (this isn't the absolute worst case, but it's far worse than the ~25% median dip in value after 2008 [recorded by zillow](http://www.zillow.com/home-values/)):

In [17]:
def buy_ohno(months, r):
    return buy_in_general(mortgage, 0.02, 0.065, 0.01, yearly_taxes, months, 0.5, r)

In [18]:
for year in xrange(1, 21):
    def dcf(r_i): return buy_ohno(12 * year, r_i / 12) - rent(12 * year, r_i / 12)
    print year, irr(dcf)

1 None
2 None
3 None
4 None
5 None
6 None
7 None
8 None
9 None
10 None
11 0.024
12 0.07
13 0.096
14 0.113
15 0.125
16 0.133
17 0.14
18 0.144
19 0.148
20 0.151


Pretty grim, but still not a doomsday. You'll need to stay in the house for a long time to beat the market if you got caught in a bad market cycle.